In [1]:
import torch
from transformers import DistilBertTokenizer, AutoModelForSequenceClassification, TrainingArguments,Trainer
import numpy as np
import pandas as pd
import evaluate
from numpy.random import RandomState
import torch.utils.data as data_utils

accuracy = evaluate.load("accuracy")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
#model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
training_args = TrainingArguments(output_dir = "test_trainer",evaluation_strategy = "epoch")

/home/willmsc/.conda/envs/mentalhealth/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceCl

In [34]:
dataset = pd.read_csv("mental_health.csv")
size = len(dataset)
print(dataset)
#rng = RandomState()
#train = dataset.sample(frac=0.8, random_state=rng)
#test = dataset.loc[~dataset.index.isin(train.index)]


#train_set = dataset.sample(frac = 0.8)
#val_set = dataset.drop(train_set.index)


training_size = int(size*.8+0.5)
training_set = data_utils.DataLoader(dataset[0:training_size].to_numpy())
val_set = data_utils.DataLoader(dataset[training_size:].to_numpy())
print(type(training_set))
test_size = size - training_size
#train_set, val_set = data_utils.random_split(dataset, [training_size,test_size])
#figure out what train_set is, use it int the trainer, test to see how things work

                                                    text      label
0      dear american teens question dutch person hear...    Healthy
1      nothing look forward lifei dont many reasons k...  Unhealthy
2      music recommendations im looking expand playli...    Healthy
3      im done trying feel betterthe reason im still ...  Unhealthy
4      worried  year old girl subject domestic physic...  Unhealthy
...                                                  ...        ...
27972  posting everyday people stop caring  religion ...    Healthy
27973  okay definetly need hear guys opinion ive pret...    Healthy
27974  cant get dog think ill kill myselfthe last thi...  Unhealthy
27975  whats point princess bridei really think like ...  Unhealthy
27976  got nudes person might might know snapchat do ...    Healthy

[27977 rows x 2 columns]
<class 'torch.utils.data.dataloader.DataLoader'>


In [22]:
def eval_model(predictions):
        predict_matrix = np.argmax(predictions[0],axis=-1)
        return accuracy.compute(predictions = predict_matrix, references = predictions[1])


In [23]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [24]:
type(training_set)

torch.utils.data.dataloader.DataLoader

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_set,
    eval_dataset = val_set,
    compute_metrics = compute_metrics
)

In [30]:
trainer.train()

***** Running training *****
  Num examples = 22382
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8394
  Number of trainable parameters = 66955010


KeyError: 18201

In [ ]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]